In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import sqlite3

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
# %%
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

BASE_URL = "https://www.camara.leg.br/noticias/noticias-institucionais"
print("✅ Scraper da Câmara (Institucionais) pronto!")

✅ Scraper da Câmara (Institucionais) pronto!


In [3]:
DATABASE_NAME = "internet_governance_news.db"

def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela 'articles' prontos!")

create_database()


✅ Banco e tabela 'articles' prontos!


In [4]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


In [5]:
# %%
def load_articles_from_db():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT *
        FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles_from_db()
display(df_db.head())
print(f"📦 Total no banco: {len(df_db)} registros")


,id,title,date,author,url,source
0,683,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,1374,CGI.br busca saídas para o acesso mundial à re...,31 OUT 2007,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-busca-s...,CGI Notícias
2,1399,ICANN 2006 debate no Brasil as tendências da i...,31 OUT 2006,CGI.br Notícias,https://cgi.br/noticia/releases/icann-2006-deb...,CGI Notícias
3,797,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
4,1140,TIC Empresas 2013,31 MAI 2014,CGI.br Notícias,https://cgi.br/noticia/notas/tic-empresas-2013/,CGI Notícias


📦 Total no banco: 2573 registros


In [6]:
# %%
def montar_url(pagina):
    if pagina == 1:
        return f"{BASE_URL}"
    return f"{BASE_URL}?pagina={pagina}"

def extrair_paragrafos(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        ps = soup.find_all("p")
        textos = [
            p.get_text(strip=True)
            for p in ps
            if len(p.get_text(strip=True).split()) > 10
        ]
        return textos[:5] if textos else ["NA"]
    except:
        return ["NA"]


In [7]:
# %%
noticias = []
TOTAL_PAGES = 78

for pagina in range(TOTAL_PAGES, 0, -1):
    url = montar_url(pagina)
    print(f"📄 Coletando página {pagina}: {url}")

    r = requests.get(url, headers=HEADERS, timeout=10)
    if r.status_code != 200:
        print("⚠️ Erro ao acessar página")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    # ===== Seletores adaptados para NOTÍCIAS INSTITUCIONAIS
    itens = soup.select("li.l-lista-noticias__item")
    print(f"   {len(itens)} notícias encontradas")

    for item in itens:
        artigo = item.select_one("article.g-chamada")
        if not artigo:
            continue

        # título & link
        titulo_tag = artigo.select_one(".g-chamada__titulo a")
        if not titulo_tag:
            continue

        titulo = titulo_tag.get_text(strip=True)
        link = titulo_tag["href"]

        # data e hora
        date_tag = artigo.select_one(".g-artigo__data-hora")
        data_raw = date_tag.get_text(strip=True) if date_tag else "NA"

        # extrai texto
        paragrafos = extrair_paragrafos(link)

        # acumula
        noticias.append({
            "titulo": titulo,
            "data": data_raw,
            "link": link,
            "paragrafos": " || ".join(paragrafos),
            "fonte": "Câmara dos Deputados"
        })

        # grava no banco
        insert_article(
            title=titulo,
            date=data_raw,
            author="Agência Câmara",
            url=link,
            source="Câmara dos Deputados"
        )

    time.sleep(1)

print(f"\n✅ Total coletado: {len(noticias)} notícias")

df_camara = pd.DataFrame(noticias)
display(df_camara.head())


📄 Coletando página 78: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=78
   8 notícias encontradas
📄 Coletando página 77: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=77
   10 notícias encontradas
📄 Coletando página 76: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=76
   10 notícias encontradas
📄 Coletando página 75: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=75
   10 notícias encontradas
📄 Coletando página 74: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=74
   10 notícias encontradas
📄 Coletando página 73: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=73
   10 notícias encontradas
📄 Coletando página 72: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=72
   10 notícias encontradas
📄 Coletando página 71: https://www.camara.leg.br/noticias/noticias-institucionais?pagina=71
   10 notícias encontradas
📄 Coletando página 70: https://www.camara.leg.br/

,titulo,data,link,paragrafos,fonte
0,Exposição mostra danos causados por invasões à...,10/01/2024 09:56,https://www.camara.leg.br/assessoria-de-impren...,Objetivo da mostra é recordar um ano da invasã...,Câmara dos Deputados
1,Câmara realiza primeiro leilão de bens móveis ...,09/01/2024 10:46,https://www.camara.leg.br/assessoria-de-impren...,A Câmara dos Deputados realiza no dia 22 de ja...,Câmara dos Deputados
2,Página e programação especial lembram 1 ano da...,08/01/2024 08:24,https://www.camara.leg.br/assessoria-de-impren...,"A Câmara divulga, a partir desta semana, uma s...",Câmara dos Deputados
3,Congresso recebe projeção das cores da bandeir...,05/01/2024 12:39,https://www.camara.leg.br/assessoria-de-impren...,O Congresso recebe projeção com as cores da Ba...,Câmara dos Deputados
4,Visitação institucional ao Congresso fica susp...,03/01/2024 15:46,https://www.camara.leg.br/assessoria-de-impren...,A visitação institucional ao Palácio do Congre...,Câmara dos Deputados


In [8]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT * FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head(20))


📦 Total no banco: 3351 registros


,id,title,date,author,url,source
0,3578,Câmara divulga Relatório de Participação Popul...,31/12/2024 09:49,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
1,3510,Câmara divulga resultado de concurso de vídeos...,31/10/2024 11:01,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
2,3785,Congresso se ilumina na cor laranja em apoio à...,31/07/2025 15:07,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
3,3389,Projeto sobre aborto é o destaque da participa...,31/07/2024 16:57,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
4,3390,Programa da Câmara voltado ao público infantoj...,31/07/2024 11:59,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
5,3644,"Autor da mostra ""A cidade se perde nas ausênci...",31/03/2025 09:08,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
6,3194,Visitação institucional ao Congresso fica susp...,31/01/2024 15:04,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
7,3195,Medalha do Mérito Legislativo de 2023 será ent...,31/01/2024 11:50,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
8,683,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
9,1374,CGI.br busca saídas para o acesso mundial à re...,31 OUT 2007,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-busca-s...,CGI Notícias


In [9]:
keywords = ['digital', 'internet', 'IA', 'tecnologia', 'dados', 'privacidade']
pattern = r'|'.join(keywords)

df_filt = df_camara[
    df_camara['titulo'].str.contains(pattern, case=False, na=False, regex=True) |
    df_camara['paragrafos'].str.contains(pattern, case=False, na=False, regex=True)
].copy()

print(f"{len(df_filt)} notícias filtradas (de {len(df_camara)})")
display(df_filt.head())


774 notícias filtradas (de 778)


,titulo,data,link,paragrafos,fonte
0,Exposição mostra danos causados por invasões à...,10/01/2024 09:56,https://www.camara.leg.br/assessoria-de-impren...,Objetivo da mostra é recordar um ano da invasã...,Câmara dos Deputados
1,Câmara realiza primeiro leilão de bens móveis ...,09/01/2024 10:46,https://www.camara.leg.br/assessoria-de-impren...,A Câmara dos Deputados realiza no dia 22 de ja...,Câmara dos Deputados
2,Página e programação especial lembram 1 ano da...,08/01/2024 08:24,https://www.camara.leg.br/assessoria-de-impren...,"A Câmara divulga, a partir desta semana, uma s...",Câmara dos Deputados
3,Congresso recebe projeção das cores da bandeir...,05/01/2024 12:39,https://www.camara.leg.br/assessoria-de-impren...,O Congresso recebe projeção com as cores da Ba...,Câmara dos Deputados
4,Visitação institucional ao Congresso fica susp...,03/01/2024 15:46,https://www.camara.leg.br/assessoria-de-impren...,A visitação institucional ao Palácio do Congre...,Câmara dos Deputados


In [10]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para gráficos")
        return

    # ------------------------------
    # Top 15
    # ------------------------------
    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x='rank',
        y='title',
        orientation='h',
        title='Top 15 Notícias – Internet Governance'
    )
    fig1.update_layout(height=600)
    fig1.show()

    # ------------------------------
    # Pizza por Fonte (BANCO)
    # ------------------------------
    # Fonte
    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # ------------------------------
    # Nuvem de Palavras
    # ------------------------------
    text = ' '.join(df['title'].astype(str)).lower()
    words = re.findall(r'\b\w{4,}\b', text)

    wc = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    wc.columns = ['palavra', 'freq']

    fig3 = px.treemap(
        wc,
        path=['palavra'],
        values='freq',
        title='Nuvem de Palavras – Títulos'
    )
    fig3.show()


In [11]:
plot_charts(df_db)